In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from taskdataset import TaskDataset
import matplotlib.pyplot as plt
import os
import sys
from endpoints.requests import sybil, sybil_reset
import sklearn
import time

In [5]:
print(os.getcwd())
dataset = torch.load(r"data\SybilAttack.pt")

c:\Users\Krzysztof\Desktop\ensembleAI-ScoutTeam\task_2_sybilattack


In [55]:
index = dataset.ids[:2000]
print(index)
sybil_reset(binary_or_affine = 'binary', home_or_defense ='home')
time.sleep(5)
api_bin_home = sybil(ids=index,
                 home_or_defense='home',
                 binary_or_affine='binary')


[101031, 8526, 43127, 191394, 298792, 121086, 149475, 102605, 163605, 101855, 135720, 202472, 78948, 205516, 130325, 61352, 12573, 148600, 288522, 121039, 142636, 248729, 98173, 110229, 251315, 294900, 69630, 246410, 205724, 241323, 239570, 74090, 206834, 248228, 277825, 177638, 178460, 195306, 57873, 239078, 142185, 240902, 77409, 114516, 158823, 257739, 162041, 19492, 105759, 300664, 233603, 213129, 28202, 154881, 56113, 218900, 77103, 166599, 97556, 73616, 236198, 166840, 77532, 3669, 191063, 5040, 57826, 74718, 304774, 48059, 21290, 268421, 236305, 229322, 280199, 305084, 107778, 230929, 272095, 141814, 297494, 262306, 7149, 222224, 66291, 104205, 252405, 231398, 45372, 292658, 144859, 201391, 1212, 270096, 287366, 208108, 71574, 260196, 210798, 162229, 292402, 170376, 193023, 6021, 135462, 122707, 298246, 171671, 263538, 76182, 141419, 256928, 293835, 13331, 217190, 19693, 122601, 187403, 35388, 189706, 56506, 130137, 51870, 41849, 81127, 88318, 277129, 254901, 100903, 100569, 195

In [58]:
sybil_reset(binary_or_affine = 'binary', home_or_defense ='defense')
time.sleep(10)
api_bina = sybil(ids=index,
                 home_or_defense='defense',
                 binary_or_affine='binary')

Request ok
{'msg': 'Successful sybil binary reset defense'}


In [59]:
bin_sybil_def = np.array(api_bina)
bin_sybil_home = np.array(api_bin_home)

In [22]:
print(f"def_shape : {bin_sybil_def.shape}, home_shape : {bin_sybil_home.shape}")

def_shape : (2000, 384), home_shape : (2000, 384)


In [63]:
def split_data(X, y, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, shuffle=True, random_seed=None):
    """
    Splits the features (X) and target labels (y) into train, validation, and test sets.

    Parameters:
    - X: numpy array, the feature matrix.
    - y: numpy array, the target vector.
    - train_ratio: float, proportion of the dataset for training (default: 0.7).
    - val_ratio: float, proportion of the dataset for validation (default: 0.15).
    - test_ratio: float, proportion of the dataset for testing (default: 0.15).
    - shuffle: bool, whether to shuffle the dataset before splitting (default: True).
    - random_seed: int, seed for the random number generator (default: None).

    Returns:
    - train_X: numpy array, training features.
    - val_X: numpy array, validation features.
    - test_X: numpy array, test features.
    - train_y: numpy array, training target labels.
    - val_y: numpy array, validation target labels.
    - test_y: numpy array, test target labels.
    """
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"
    assert X.shape[0] == y.shape[0], "Number of samples in X and y must be the same"
    
    # Combine X and y for shuffling
    data = np.column_stack((X, y))

    # Set random seed if provided
    if random_seed:
        np.random.seed(random_seed)

    # Shuffle the dataset if required
    if shuffle:
        np.random.shuffle(data)

    # Compute sizes of each split
    num_samples = data.shape[0]
    train_size = int(train_ratio * num_samples)
    val_size = int(val_ratio * num_samples)

    x_len, y_len = data.shape
    # Split the dataset
    train_X, train_y = data[:train_size, :y_len//2], data[:train_size, y_len//2:]
    val_X, val_y = data[train_size:train_size + val_size, :y_len//2], data[train_size:train_size + val_size, y_len//2:]
    test_X, test_y = data[train_size + val_size:, :y_len//2], data[train_size + val_size:, y_len//2:]

    return train_X, val_X, test_X, train_y, val_y, test_y

# Example usage:
# Assuming 'X' is your feature matrix and 'y' is your target vector
X_train, X_val, X_test, y_train, y_val, y_test = split_data(bin_sybil_def, bin_sybil_home,train_ratio=0.8,val_ratio = 0., test_ratio = 0.2)
#train_data_def, val_data_def, test_data_def = split_data(bin_sybil_def)


In [64]:
print(f"def_shape : {X_train.shape}, home_shape : {y_train.shape}")

def_shape : (1600, 384), home_shape : (1600, 384)


In [27]:
class RBF:
    def __init__(self, num_centers, sigma):
        self.num_centers = num_centers
        self.sigma = sigma
        self.centers = None
        self.weights = None

    def _gaussian(self, x, c):
        return np.exp(-np.linalg.norm(x - c) ** 2 / (2 * self.sigma ** 2))

    def _design_matrix(self, X):
        return np.array([self._gaussian(x, c) for c in self.centers] for x in X)

    def fit(self, X, y):
        self.centers = X[np.random.choice(len(X), self.num_centers, replace=False)]
        design_matrix = self._design_matrix(X)
        self.weights = np.linalg.lstsq(design_matrix, y, rcond=None)[0]

    def predict(self, X):
        design_matrix = self._design_matrix(X)
        return design_matrix.dot(self.weights)


In [24]:
# Example usage
X_train = np.random.rand(100, 2)  # Example training data
y_train = np.sin(X_train[:, 0]) + np.cos(X_train[:, 1])  # Example training labels

rbf = RBF(num_centers=10, sigma=1.0)
rbf.fit(X_train, y_train)

X_test = np.random.rand(10, 2)  # Example test data
predictions = rbf.predict(X_test)
print(predictions)


(1400, 384)

In [2]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        #self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size1, output_size)
    
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        #x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

In [3]:
from sklearn.neural_network import MLPRegressor
from torch.utils.data import DataLoader, TensorDataset

In [123]:
model_reg =  MLPRegressor(hidden_layer_sizes=(10,10,10,),random_state=85, max_iter=5000).fit(X_train, y_train)
model = MLP(384,512,384)


In [114]:
#X_test_sm, y_test_sm = X_test[:200], y_test[:200]
X_train_sm, y_train_sm = X_train[:200], y_train[:200] 
#print(f"shapes {X_test_sm.shape,y_test_sm.shape}")
model_reg_small =  MLPRegressor(hidden_layer_sizes=(512,512,),random_state=85, max_iter=5000).fit(X_train_sm, y_train_sm)

y_pred = model_reg_small.predict(bin_sybil_def)

In [101]:
print(f"Big model: {model_reg.score(X_test, y_test)}, small model: {model_reg_small.score(X_test, y_test)}")

Big model: 0.04956420042377787, small model: -0.0570681999287509


In [115]:
print(np.mean(np.array([np.linalg.norm(y_pred-y) for y_pred, y in zip(y_pred,bin_sybil_home)])))

5.2467848077503865


In [127]:
# Define the L1 regularization penalty
def l1_penalty(model, l1_lambda):
    l1_reg = torch.tensor(0., requires_grad=True)
    for name, param in model.named_parameters():
        if 'weight' in name:
            l1_reg = l1_reg + torch.norm(param, 1)
    return l1_lambda * l1_reg

In [202]:
wyn =[]
index = np.array([1,2,3])
index2 = np.array([3,4,5])
out = [[1,2,3],[2,3,4],[4,5,6]]
out2 = [[1,2,3],[2,3,4],[4,5,6]]
#data = np.hstack((index, out))
a=np.concatenate((wyn,index,index2))
#outr = np.concatenate((wyn,out,out2))
#outr
a

array([1., 2., 3., 3., 4., 5.])

In [9]:
sybil_reset(binary_or_affine = 'binary', home_or_defense ='defense')
sybil_reset(binary_or_affine = 'binary', home_or_defense ='home')
inds = []
represen = []

for i in range(10):
    print(f"epoch: {i}")
    index = dataset.ids[i*2000:(i+1)*2000]
    sybil_reset(binary_or_affine = 'binary', home_or_defense ='defense')
    time.sleep(5)
    index_home = index[:200]
    api_bin_home = np.array(sybil(ids=index_home,
                    home_or_defense='home',
                    binary_or_affine='binary'))
    time.sleep(10)
    api_bina = np.array(sybil(ids=index,
                    home_or_defense='defense',
                    binary_or_affine='binary'))
    

    model = MLP(384,600,0,384)
    X_train_sm, y_train_sm = api_bina[:200], api_bin_home[:200]
    #X_test_sm, y_test_sm = bin_sybil_def[200:], bin_sybil_home[200:]

    '''
    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    '''
    #X_test_sm_t, y_test_sm_t = torch.tensor(X_test_sm, dtype=torch.float32),torch.tensor(y_test_sm, dtype=torch.float32)


    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_sm, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_sm, dtype=torch.float32)

    #bin_sybil_def_tensor =torch.tensor(bin_sybil_def, dtype=torch.float32)
    #bin_sybil_home_tensor =torch.tensor(bin_sybil_home, dtype=torch.float32)

    #print(X_train.shape)
    #print(X_train_tensor.shape)

    # Create DataLoader for batch training
    #train_dataset = TensorDataset(X_train_tensor.reshape((384,1600)), y_train_tensor.reshape((384,1600)))
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size =32, shuffle=True)
    
    api_bina_t = torch.tensor(api_bina)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_MSE = nn.MSELoss()

    # Training loop
    num_epochs = 60
    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader):
            model.train()
            inputs, labels = data
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss_tr = loss_MSE(outputs, labels) #+ l1_penalty(model, 1e-7)
            
            # Backward pass and optimize
            loss_tr.backward()
            optimizer.step()
            
        
        """
        if (epoch+1) % 10 == 0:  # Print every 10 epoch
            model.eval()

            with torch.inference_mode():
                out = model(bin_sybil_def_tensor)
                MSE = loss_MSE(out,bin_sybil_home_tensor)

                out_t = model(X_test_sm_t)
                MSE_t = loss_MSE(out_t,y_test_sm_t)
            print(f"Epoch [{epoch+1}/{num_epochs}], loss whole: {MSE}, loss test: {MSE_t}, train_loss : {loss_tr}]")
        """
        out = model(api_bina_t)
        if not inds:
            inds = index
        else:
            inds = np.concatenate((inds,index))
        
        if not represen:
            represen = out.detach().numpy()
        else:
            represen =np.concatenate((represen,out.detach().numpy()))
        

Request ok
{'msg': 'Successful sybil binary reset defense'}
Request ok
{'msg': 'Successful sybil binary reset home'}
epoch: 0
Request ok
{'msg': 'Successful sybil binary reset defense'}


TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [179]:
data = np.load('data/example_submission.npz')
data['representations'].shape

(20000, 192)

In [ ]:
    np.savez(
        "sybilattack/data/example_submission.npz",
        ids=np.random.permutation(20000),
        representations=np.random.randn(20000, 192),
    )

epoch = 40, loss ~ 0.07, architect 384,600,384

In [ ]:
l1_penalty = l1_weight * sum([p.abs().sum() for p in net.hidden.parameters()])